In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium
import os
import datetime

In [2]:
file_list = os.listdir('C:\JanbogoProject\JangbogoProject\data\\2019_data')
path = 'C:\JanbogoProject\JangbogoProject\data\\2019_data'

In [3]:
#너무 커서 안됨
# df_list=[]
# for filename in file_list:
#     df = pd.read_csv(path + '\\' +  filename)
#     df_list.append(df)
# data = pd.concat(df_list, sort=False).reset_index(drop=True) 

In [4]:
#통합데이터에서 가져오기로...
df = pd.read_csv('C:\JanbogoProject\JangbogoProject\develop\\target_pum_kind.csv')

C:\Users\song\anaconda3\envs\JanbogoProject2\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df1 = df.drop('Unnamed: 0',axis=1)
df1

,SALEDATE,PUM_NM,KIND_NM,DAN_NM,SAN_NM,TOT_QTY,TOT_AMT
0,20160102,건고추,기타건고추,kg,전북 익산시,3.0,33000.0
1,20160102,깻잎,깻단,kg,전남 곡성군,62.0,182550.0
2,20160102,깻잎,깻단,kg,전남 광양시,27.0,179500.0
3,20160102,깻잎,깻잎,kg,전남 곡성군,253.0,4027800.0
4,20160102,깻잎,깻잎,kg,전남 광양시,312.0,4969300.0
...,...,...,...,...,...,...,...
3402537,2020-09-28,호박,애호박,kg,충청북도 옥천군,1486.0,3030200.0
3402538,2020-09-28,호박,애호박,kg,충청북도 음성군,18528.0,55185100.0
3402539,2020-09-28,호박,애호박,kg,충청북도 진천군,10980.0,25190900.0
3402540,2020-09-28,호박,애호박,kg,충청북도 청원군,30008.0,88470500.0


In [6]:
df2 = df1.drop('SAN_NM', axis = 1)
df2

,SALEDATE,PUM_NM,KIND_NM,DAN_NM,TOT_QTY,TOT_AMT
0,20160102,건고추,기타건고추,kg,3.0,33000.0
1,20160102,깻잎,깻단,kg,62.0,182550.0
2,20160102,깻잎,깻단,kg,27.0,179500.0
3,20160102,깻잎,깻잎,kg,253.0,4027800.0
4,20160102,깻잎,깻잎,kg,312.0,4969300.0
...,...,...,...,...,...,...
3402537,2020-09-28,호박,애호박,kg,1486.0,3030200.0
3402538,2020-09-28,호박,애호박,kg,18528.0,55185100.0
3402539,2020-09-28,호박,애호박,kg,10980.0,25190900.0
3402540,2020-09-28,호박,애호박,kg,30008.0,88470500.0


In [7]:
df2.SALEDATE.unique()

array([20160102, 20160104, 20160105, ..., '20200731', '20200801',
       '20200803'], dtype=object)

In [8]:
df3 = df2.groupby(['SALEDATE','PUM_NM','KIND_NM','DAN_NM'])['TOT_QTY','TOT_AMT'].sum().reset_index()

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_24820/658758426.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df2.groupby(['SALEDATE','PUM_NM','KIND_NM','DAN_NM'])['TOT_QTY','TOT_AMT'].sum().reset_index()


In [9]:
#날짜 데이터 동일화
df1.SALEDATE = df1.SALEDATE.apply(lambda x: str(x) if '-' not in str(x) else str(x).replace('-',''))

In [10]:
#2019년 데이터만 뽑아내기
df_2019 = df1[df1.SALEDATE.apply(lambda x : True if x[:4] == '2019' else False)]

In [11]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [12]:
joosan_dict = dict()

# 품종별 주산지 mapping
for pum in unique_pum :
    pum_df = df_2019[df_2019['PUM_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break

# 품종별 주산지 mapping
for pum in unique_kind :
    pum_df = df_2019[df_2019['KIND_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break

In [13]:
joosan_dict

{'배추': '전라남도 해남군',
 '무': '제주도 제주시',
 '양파': '전라남도 무안군',
 '건고추': '전남 해남군',
 '마늘': '경상남도 창녕군',
 '대파': '전라남도 진도군',
 '얼갈이배추': '경기도 포천시',
 '양배추': '제주도 제주시',
 '깻잎': '경상남도 밀양시',
 '시금치': '경기도 포천시',
 '미나리': '경기도 시흥시',
 '당근': '제주도 제주시',
 '파프리카': '강원도 철원군',
 '새송이': '충청남도 천안시',
 '팽이버섯': '경상북도 청도군',
 '토마토': '부산 강서구',
 '청상추': '전북 남원시',
 '백다다기': '충청남도 천안시',
 '애호박': '경상남도 진주시',
 '캠벨얼리': '경상북도 상주시',
 '샤인마스캇': '경상북도 김천시'}

### 주산지 위도, 경도 구하기

In [14]:

y = [] #위도
x = [] #경도

Kakao_ServiceKey = open('Kakao_ServiceKey.txt', 'r').read()
# print(Kakao_ServiceKey)
headers = {"Authorization":'KakaoAK ' + Kakao_ServiceKey}
for san in tqdm(joosan_dict.values()) :
    # print(san)
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + san
    result = json.loads(str(requests.get(url, headers=headers).text))
    # print(result)
    match_first = result['documents'][0]['address']
    y.append(float(match_first['y'])) #위도
    x.append(float(match_first['x'])) #경도
joosan_xy = pd.DataFrame({'SAN_NM' : joosan_dict.values(), 'y' : y, 'x' : x}).reset_index()


100%|██████████| 21/21 [00:02<00:00,  9.97it/s]


In [15]:
# from urllib.parse import urlparse

In [16]:
# Kakao_ServiceKey = open('Kakao_ServiceKey.txt', 'r').read()
# Kakao_ServiceKey
# for san in tqdm(joosan_dict.values()) :
#     url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + san
#     result = requests.get(urlparse(url).geturl(), headers= {'Authorization' : 'KakaoAK ' + Kakao_ServiceKey}).json()
#     print(result)

In [17]:
#주산지 위도와 경도
joosan_xy

,index,SAN_NM,y,x
0,0,전라남도 해남군,34.573517,126.599270
1,1,제주도 제주시,33.499534,126.531171
2,2,전라남도 무안군,34.990341,126.481705
3,3,전남 해남군,34.573517,126.599270
4,4,경상남도 창녕군,35.544615,128.492168
5,5,전라남도 진도군,34.486829,126.263446
6,6,경기도 포천시,37.894993,127.200333
7,7,제주도 제주시,33.499534,126.531171
8,8,경상남도 밀양시,35.503817,128.746608
9,9,경기도 포천시,37.894993,127.200333


### 농업 기상 관측 지점 데이터

In [18]:

stn_info = pd.read_csv('C:\JanbogoProject2\JangbogoProject\data\RDA_SPOT_INFO_.csv')
# stn_info
stn_info['관측시작일'] = pd.to_datetime(stn_info['관측시작일'])
stn_info = stn_info[stn_info['관측시작일'].dt.year<=2015]

In [19]:
stn_info

,도명,지점명,지점코드,위도,경도,고도,설치주소,관측시작일
0,경기도,가평군 가평읍,477802A001,37.84621,127.50063,80.0,경기도 가평군 가평읍 아랫마장길 59,2011-05-22
1,경기도,고양시 구산동,411801A001,37.67453,126.70070,24.0,경기도 고양시 일산서구 구산동 1942,2010-12-06
2,경기도,고양시 덕양구,412040A002,37.64918,126.87036,39.0,경기도 고양시 덕양구 고양대로 1695 (원흥동),2013-08-24
3,경기도,광주시 목현동,464030A001,37.43231,127.23394,91.0,경기도 광주시 이배재로 209-5,2010-11-01
4,경기도,김포시 월곶면,415743A001,37.69489,126.55614,43.0,경기도 김포시 월곶면 오리정로 13,2010-12-14
...,...,...,...,...,...,...,...,...
205,부산,부산시 강서구,618803A001,35.20937,128.96128,3.0,경남 부산시 강서구 대저1동 2038,2010-12-20
206,인천,옹진군 백령면,409911A001,37.95742,124.71063,11.0,인천광역시 옹진군 백령면 진촌리 2359-4,2014-12-18
207,인천,옹진군 영흥면,409871A001,37.25288,126.46034,11.0,인천광역시 옹진군 영흥면 영흥남로435번길 29,2012-05-26
208,울산,울산시 청량면,689861A001,35.52411,129.26704,51.0,울산광역시 울주군 청량면 청량천변로 211,2009-03-02


### 주산지별로 농업기상관측지점과 거리 기준으로 mapping

In [20]:

y_san = joosan_xy['y']
x_san = joosan_xy['x']
y_stn = stn_info['위도']
x_stn = stn_info['경도']
code_dict = dict()

for y_san, x_san, san_name in zip(joosan_xy['y'], joosan_xy['x'], joosan_xy['SAN_NM']) :
    min_distance = 1000 #임의로 초기값 설정
    for y_stn, x_stn, stn_code in zip(stn_info['위도'], stn_info['경도'], stn_info['지점코드']) :
        distance = ((y_san-y_stn)**2 + (x_san-x_stn)**2)**0.5 #거리
        if distance < min_distance :
            min_distance = distance 
            stn_nearby = str(stn_code)
    code_dict[san_name] = stn_nearby

### 농업기상관측지점 및 산지 분포 시각화

In [21]:
stn_location = stn_info[['지점명','위도', '경도']]
san_location = joosan_xy

map = folium.Map(location = [36, 128], zoom_start =7)

# 농업기상관측지점 분포 (파랑)
for index in stn_location.index:
    stn_latitude = stn_location.loc[index,"위도"]
    stn_longtitude = stn_location.loc[index,"경도"]
    stn_tooltip = stn_location.loc[index,'지점명']
    folium.Marker([stn_latitude, stn_longtitude], popup = '('+str(stn_latitude)+', '+str(stn_longtitude)+')', tooltip = stn_tooltip).add_to(map) 

# 산지 분포 (빨강)    
for index in san_location.index:    
    san_latitude = san_location.loc[index,"y"]
    san_longtitude = san_location.loc[index,"x"]
    san_tooltip = san_location.loc[index,'SAN_NM']    
    folium.Marker([san_latitude, san_longtitude], popup = '('+str(san_latitude)+', '+str(san_longtitude)+')', tooltip = san_tooltip, icon = folium.Icon(color = 'red')).add_to(map)
    
map

### 농업기상데이터 API - 월별 일 기본 관측데이터 조회

In [22]:
### https://www.data.go.kr/iim/api/selectAPIAcountView.do
### 농촌진흥청 국립농업과학원_농업기상 기본 관측데이터 조회 

In [23]:
CropWeather_ServiceKey = open('CropWeather_ServiceKey.txt', 'r').read()
year_list = ['2015','2016', '2017', '2018', '2019', '2020']
month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
weather = pd.DataFrame()
first_run = 0
code_list = code_dict.values() # 주산지에 mapping 된 지점에 대해서만 조회
year_error, month_error, stn_code_error, url_error, f_obs_date_error = [],[],[],[],[]
colname_dict = dict()

for stn_code in tqdm(code_list) :
    for year in year_list :
        for month in month_list :
            url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherMonDayList?'
            params = {
                'serviceKey' : CropWeather_ServiceKey, #인증키
                'Page_No' : '1', # 페이지 번호
                'Page_Size' : '31', # 한 페이지 결과 수(1~100) (31일 이내 전체 표기)
                'search_Year' : year, # 관측년도
                'search_Month' : month, #관측월
                'obsr_Spot_Code' : stn_code # 관측지점코드
            }

            # url에 params 적용하기(붙이기)
            for key, value in zip(params.keys(), params.values()):
                if key == 'serviceKey' :
                    url = url + key +'=' + value
                else :
                    url = url + '&' + key + '=' + value
            
            try :
                response = urllib.request.urlopen(url).read()
                response_string = ET.fromstring(response)

                # response - header(0) / body(1) - ...items(3) - item(0) 
                items = response_string[1][3]

                if first_run == 0 :
                    for i in items[0] :
                        colname_dict[i.tag] = [] # {'no' : [], 'stn_Code' : [], ...} 
                        first_run += 1

                # 일자별로 반복 실행
                num_days = len(items) #28 or 30 or 31
                for index in range(num_days) : 
                    # 해당하는 리스트에 원소 넣기
                    for i in items[index] :
                        colname_dict[i.tag].append(i.text)
            except :
                year_error.append(year)
                month_error.append(month)
                stn_code_error.append(stn_code)
                url_error.append(url)
                f_obs_date_error.append(stn_info[stn_info['지점코드']==stn_code].reset_index()['관측시작일'][0])
               
            
# DataFrame에 값 채워넣기             
for col in colname_dict.keys() :
    weather[col] = colname_dict[col]

# 에러 발생한 요청 모음    
error = pd.DataFrame({'year': year_error, 
                      'month': month_error,
                      'stn_code': stn_code_error,
                      'url': url_error,  
                      '관측시작일': f_obs_date_error})

100%|██████████| 17/17 [02:13<00:00,  7.87s/it]


In [24]:
error

,year,month,stn_code,url,관측시작일
0,2015,01,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
1,2015,02,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
2,2015,03,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
3,2015,04,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
4,2015,05,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
...,...,...,...,...,...
1087,2020,08,037268B004,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2007-03-20
1088,2020,09,037268B004,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2007-03-20
1089,2020,10,037268B004,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2007-03-20
1090,2020,11,037268B004,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2007-03-20


In [25]:
# 중복 제거
# weather = weather.drop_duplicates().reset_index(drop=True)
# weather
weather = pd.read_csv('C:\JanbogoProject2\JangbogoProject\data\weather_op.csv')
weather

,no,stn_Code,stn_Name,date,temp,max_Temp,min_Temp,hum,widdir,wind,rain,sun_Time,sun_Qy,condens_Time,gr_Temp,soil_Temp,soil_Wt
0,1,536824B002,해남군 옥천면,2015-01-01,-1.3,0.6,-2.9,80.0,295.2,2.3,0.8,NaN,7.8,NaN,NaN,3.36,25.9
1,1,330846A001,천안시 목천읍,2015-01-01,-6.2,-3.8,-8.3,NaN,NaN,0.0,0.0,NaN,NaN,1429.0,NaN,NaN,NaN
2,1,627911A001,밀양시 상남면,2015-01-01,-3.2,0.2,-7.2,40.1,282.7,2.9,0.0,516.0,11.0,0.0,NaN,2.20,28.5
3,1,539823A001,진도군 군내면,2015-01-01,-0.8,1.6,-2.8,79.2,257.0,3.5,1.5,217.0,8.2,652.0,NaN,5.02,30.6
4,1,590823A001,남원시 이백면,2015-01-01,-4.1,-1.3,-6.0,60.7,286.7,2.1,0.5,310.0,7.7,0.0,-4.3,2.16,20.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34258,31,590823A001,남원시 이백면,2020-12-31,-2.6,-0.3,-5.8,74.6,232.0,1.0,0.0,339.0,9.7,87.0,-2.6,7.23,22.8
34259,31,5348330,무안군 청계면,2020-12-31,-3.7,0.9,-8.3,95.3,353.5,1.5,1.0,215.0,8.2,0.0,-1.2,2.50,18.6
34260,31,330846A001,천안시 목천읍,2020-12-31,-9.9,-4.2,-16.4,70.3,171.9,0.1,0.5,NaN,30.4,0.0,-10.4,1.03,13.3
34261,31,269811A001,철원군 동송읍,2020-12-31,-11.2,-5.5,-16.9,43.0,250.6,2.3,0.0,NaN,8.3,0.0,-5.7,-4.95,22.5


In [26]:
weather[weather.stn_Code.apply(lambda x : True if len(x) < 10 else False)]

,no,stn_Code,stn_Name,date,temp,max_Temp,min_Temp,hum,widdir,wind,rain,sun_Time,sun_Qy,condens_Time,gr_Temp,soil_Temp,soil_Wt
39,1,5348330,무안군 청계면,2015-01-03,3.3,6.4,0.7,71.3,42.2,0.2,0.0,222.0,3.8,NaN,2.8,3.77,5.0
51,2,5348330,무안군 청계면,2015-01-04,4.2,11.5,-0.7,82.6,17.9,0.6,0.0,477.0,9.9,NaN,3.2,4.00,5.0
65,3,5348330,무안군 청계면,2015-01-05,6.5,13.0,-1.3,85.2,85.8,0.8,10.5,203.0,7.5,NaN,5.6,4.43,5.0
70,4,5348330,무안군 청계면,2015-01-06,4.0,13.5,-0.1,72.2,309.8,2.7,0.0,467.0,12.5,NaN,3.3,5.64,5.0
94,5,5348330,무안군 청계면,2015-01-07,-0.1,3.0,-3.6,65.1,335.9,2.0,0.0,421.0,10.2,NaN,-0.7,3.07,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34197,27,5348330,무안군 청계면,2020-12-27,5.8,7.4,2.2,86.1,58.1,1.5,9.5,0.0,1.9,0.0,5.2,5.02,15.4
34213,28,5348330,무안군 청계면,2020-12-28,4.1,12.7,-0.5,96.2,40.2,0.8,0.5,454.0,10.9,0.0,4.6,5.44,16.7
34220,29,5348330,무안군 청계면,2020-12-29,3.1,9.1,-0.1,98.0,2.5,1.8,2.5,44.0,3.4,0.0,3.0,4.92,17.4
34234,30,5348330,무안군 청계면,2020-12-30,-3.1,0.4,-5.4,91.9,322.1,3.5,0.0,123.0,7.1,0.0,-1.7,2.69,18.2


In [27]:
# dtype변환(object --> float)
for col in weather.columns[4:] :
    weather[col] = weather[col].astype(float)

### 전처리 (기상변수로 추가)
##### 30일씩 12쿼터로 나누어서 평균 기온, 평균 습도, 누적 강수량, 이상 기후 누적 일수 등 추가

In [28]:
first_date = datetime.datetime.strptime('2020-09-28', '%Y-%m-%d') - datetime.timedelta(360)
date_list = [] 
for delta in range(360) :
    date = first_date + datetime.timedelta(days = delta)
    date = datetime.datetime.strftime(date, '%Y-%m-%d')
    date_list.append(date)
date_df = pd.DataFrame({'date' : date_list})
train = pd.read_csv('C:\JanbogoProject2\JangbogoProject\data\\train.csv')
train2 = pd.concat([date_df, train], sort = False).reset_index(drop=True) #2015~2020-09-28

In [29]:
def weather_feature(temp_df, train2, date_df, allweather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12) :
    # 2015년도 일자 추가
    temp_df = train2[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    
    # 품종과 주산지 날씨 mapping
    joosanji = joosan_dict[pum] # 품종, 품목 : 주산지 매칭 딕셔너리
    joosan_code = code_dict[joosanji] #주산지 : 관측소 코드 매칭 딕셔너리
    joosan_weather = allweather[allweather['stn_Code']==joosan_code].reset_index(drop=True) # 관측소별 날씨데이터
    try: 
        end_index = np.where(joosan_weather['date']=='2020-09-28')[0][0]
    except:
        print(joosan_weather['date'])
        # end_index = np.where(joosan_weather['date']=='2020-09-27')[0][0]
        print('Out of index!!!!!!!!!!!!','pum')
        return
        # end_index = np.where(joosan_weather['date']=='2020-09-27')[0][0]
    joosan_weather = joosan_weather.iloc[:end_index+1] #2020-09-28 까지만 자르기
    temp_df = temp_df.merge(joosan_weather, on='date', how='left')

    # weather feature 추가
    col_list = temp_df.columns[6:]
    for num in range(1,num_quaters+1) :
        for index in range(360, len(temp_df)) :
            temp_quater_df = temp_df.iloc[index-quater_days*num : index-quater_days*(num-1)] #
            quater_temp = temp_quater_df['temp']
            temp_df.loc[index, f'rain_sum_{num}q'] = temp_quater_df['rain'].sum() # 누적 강수량
            temp_df.loc[index, f'heavy_rain_count_{num}q'] = np.where(temp_quater_df['rain']>90, 1, 0).sum() # 평균 강수량 90mm 이상 누적 일수
            temp_df.loc[index, f'low_temp_count_{num}q'] = np.where(quater_temp<5, 1, 0).sum() # 일평균 기온 5도 이하 누적 일수
            temp_df.loc[index, f'middle_temp_count_{num}q'] = np.where(((quater_temp>15)&(quater_temp<22)), 1, 0).sum() # 일평균 기온 15~22도 누적 일수
            temp_df.loc[index, f'high_temp_count_{num}q'] = np.where(quater_temp>32, 1, 0).sum() # 일평균 기온 32도 이상 누적 일수
            for col in col_list :
                temp_df.loc[index, f'avg_{col}_{num}q'] = temp_quater_df[col].mean() # 각 기상 요소의 평균값
    
    drop_col_list = temp_df.columns[3:19]
    temp_df = temp_df.drop(drop_col_list, 1).reset_index(drop=True)
    temp_df = temp_df.iloc[360:].reset_index(drop=True)
    
    return temp_df

In [30]:
# unique_pum = [
#     '배추', '무', '양파', '건고추','마늘',
#     '대파', '얼갈이배추', '양배추', '깻잎',
#     '시금치', '미나리', '당근',
#     '파프리카', '새송이', '팽이버섯', '토마토',
# ]

# unique_kind = [
#     '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
# ]

In [31]:
# weather_feature 함수 예시
pum_df_list = []
for pum in tqdm(unique_pum):
    print(pum)
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_pum_df = weather_feature(temp_df, train2, date_df, weather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12)
    pum_df_list.append(temp_pum_df)
    print(pum,'done!!!!!')
    print('--------------------')

# kind_df_list = []
# for kind in unique_kind:
# pum_kind_df_list = pum_df_list.extend(kind_df_list)
# pd.concat(pum_kind_df_list, axis = 0)

  0%|          | 0/16 [00:00<?, ?it/s]

배추


C:\Users\song\anaconda3\envs\JanbogoProject2\lib\site-packages\pandas\core\indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_24820/1433559709.py:35: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  temp_df = temp_df.drop(drop_col_list, 1).reset_index(drop=True)
  6%|▋         | 1/16 [01:50<27:33, 110.23s/it]

배추 done!!!!!
--------------------
무


 12%|█▎        | 2/16 [03:41<25:48, 110.59s/it]

...

새송이 done!!!!!
--------------------
팽이버섯


 94%|█████████▍| 15/16 [27:37<01:50, 110.34s/it]

팽이버섯 done!!!!!
--------------------
토마토


100%|██████████| 16/16 [29:30<00:00, 110.68s/it]

토마토 done!!!!!
--------------------


In [32]:
# weather_feature 함수 예시
for kind in tqdm(unique_kind):
    print()
    temp_df = train[['date',f'{kind}_거래량(kg)', f'{kind}_가격(원/kg)']]
    temp_kind_df = weather_feature(temp_df, train2, date_df, weather, kind, joosan_dict, code_dict, quater_days=30, num_quaters=12)
    
    pum_df_list.append(temp_kind_df)
    print(kind,'done!!!!!')
    print('--------------------')


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_24820/1433559709.py:35: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  temp_df = temp_df.drop(drop_col_list, 1).reset_index(drop=True)
 20%|██        | 1/5 [01:50<07:22, 110.63s/it]

청상추 done!!!!!
--------------------



 40%|████      | 2/5 [03:41<05:31, 110.63s/it]

백다다기 done!!!!!
--------------------



...

애호박 done!!!!!
--------------------



 80%|████████  | 4/5 [07:21<01:50, 110.28s/it]

캠벨얼리 done!!!!!
--------------------



100%|██████████| 5/5 [09:12<00:00, 110.52s/it]

샤인마스캇 done!!!!!
--------------------


In [33]:
# pum = '양파'
# temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
# temp_pum_df = weather_feature(temp_df, train2, date_df, weather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12)
# pum_df_list[2] = temp_pum_df
# temp_pum_df

In [34]:
pum_df_list2  = pum_df_list

In [35]:
# #품종 품목별로 뽑아낸 날씨 데이터를 모두 합치기 및 컬럼 명 변경
unique_pum_kind = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]
# test = pd.concat(pum_df_list, axis = 0)
# # # pum_df_list[20]

def col_name_change(col_list,unique_pum_kind,index):
    result = col_list[:2]
    for i in range(2,len(col_list)):
        temp = col_list[i] + '_' + unique_pum_kind[index]
        result.append(temp) 
    return result

for i in range(len(pum_df_list2)):
    pum_df_list2[i].columns = col_name_change(list(pum_df_list2[i].columns),unique_pum_kind,i)

test = pum_df_list2[0]

for i in tqdm(range(0,len(pum_df_list2)-1)):
    print(pum_df_list2[i+1].shape, i+1)
    # pum_df_list[i+1].columns = col_name_change(list(pum_df_list[i+1].columns),unique_pum_kind,i+1)
    temp_merge= pd.merge(test, pum_df_list2[i+1], on = 'date', how = 'left')
    test = temp_merge

test

  0%|          | 0/20 [00:00<?, ?it/s]

(1733, 219) 1
(1733, 219) 2
(1733, 219) 3
(1733, 219) 4
(1733, 219) 5
(1733, 219) 6
(1733, 219) 7
(1733, 219) 8


 40%|████      | 8/20 [00:00<00:00, 77.85it/s]

(1733, 219) 9
(1733, 219) 10
(1733, 219) 11
(1733, 219) 12
(1733, 219) 13
(1733, 219) 14


 80%|████████  | 16/20 [00:00<00:00, 54.09it/s]

(1733, 219) 15
(1733, 219) 16
(1733, 219) 17
(1733, 219) 18


100%|██████████| 20/20 [00:00<00:00, 49.65it/s]

(1733, 219) 19
(1733, 219) 20


,date,배추_거래량(kg),배추_가격(원/kg)_배추,rain_sum_1q_배추,heavy_rain_count_1q_배추,low_temp_count_1q_배추,middle_temp_count_1q_배추,high_temp_count_1q_배추,avg_temp_1q_배추,avg_max_Temp_1q_배추,...,avg_hum_12q_샤인마스캇,avg_widdir_12q_샤인마스캇,avg_wind_12q_샤인마스캇,avg_rain_12q_샤인마스캇,avg_sun_Time_12q_샤인마스캇,avg_sun_Qy_12q_샤인마스캇,avg_condens_Time_12q_샤인마스캇,avg_gr_Temp_12q_샤인마스캇,avg_soil_Temp_12q_샤인마스캇,avg_soil_Wt_12q_샤인마스캇
0,2016-01-01,0.0,0.0,249.5,0.0,0.0,21.0,0.0,21.116667,26.186667,...,86.020000,185.826667,0.403333,0.350000,381.600000,13.110000,516.766667,NaN,18.640333,19.090000
1,2016-01-02,80860.0,329.0,233.1,0.0,1.0,21.0,0.0,20.313333,25.496667,...,85.700000,188.796667,0.406667,0.350000,381.800000,12.956667,517.633333,NaN,18.417000,18.673333
2,2016-01-03,0.0,0.0,232.6,0.0,1.0,21.0,0.0,19.596667,24.846667,...,85.666667,183.560000,0.396667,0.350000,391.966667,12.980000,525.800000,NaN,18.191667,18.286667
3,2016-01-04,1422742.5,478.0,232.6,0.0,1.0,21.0,0.0,18.903333,24.323333,...,85.856667,189.790000,0.396667,0.350000,403.000000,13.066667,549.866667,NaN,17.968667,17.910000
4,2016-01-05,1167241.0,442.0,232.6,0.0,1.0,21.0,0.0,18.240000,23.700000,...,85.230000,188.196667,0.406667,0.100000,419.633333,13.396667,537.600000,NaN,17.757667,17.510000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,2020-09-24,1856965.0,1839.0,321.0,0.0,0.0,17.0,0.0,22.106667,26.833333,...,87.440000,183.740000,0.406667,6.633333,344.000000,12.600000,534.866667,NaN,19.543333,20.426667
1729,2020-09-25,1880095.5,1789.0,321.5,0.0,0.0,18.0,0.0,21.826667,26.580000,...,87.116667,188.806667,0.416667,6.633333,347.700000,12.540000,535.866667,NaN,19.323000,20.186667
1730,2020-09-26,1661090.9,1760.0,247.5,0.0,0.0,19.0,0.0,21.613333,26.530000,...,86.676667,189.903333,0.433333,6.633333,366.866667,12.900000,550.400000,NaN,19.084333,19.950000
1731,2020-09-27,25396.0,3066.0,249.5,0.0,0.0,20.0,0.0,21.363333,26.390000,...,86.053333,191.553333,0.433333,1.433333,385.033333,13.333333,520.366667,NaN,18.857333,19.580000


In [52]:
pum_df_list[4]


,date,마늘_거래량(kg),마늘_가격(원/kg)_마늘,rain_sum_1q_마늘,heavy_rain_count_1q_마늘,low_temp_count_1q_마늘,middle_temp_count_1q_마늘,high_temp_count_1q_마늘,avg_temp_1q_마늘,avg_max_Temp_1q_마늘,...,avg_hum_12q_마늘,avg_widdir_12q_마늘,avg_wind_12q_마늘,avg_rain_12q_마늘,avg_sun_Time_12q_마늘,avg_sun_Qy_12q_마늘,avg_condens_Time_12q_마늘,avg_gr_Temp_12q_마늘,avg_soil_Temp_12q_마늘,avg_soil_Wt_12q_마늘
0,2016-01-01,0.0,0.0,255.0,0.0,0.0,18.0,0.0,21.576667,26.560000,...,76.290000,134.043333,1.390000,0.550000,431.333333,12.886667,117.433333,15.233333,19.275333,22.406667
1,2016-01-02,15019.0,5475.0,254.0,0.0,1.0,18.0,0.0,20.626667,25.806667,...,76.096667,129.243333,1.393333,0.550000,432.500000,12.720000,117.933333,14.946667,19.025333,22.180000
2,2016-01-03,0.0,0.0,254.0,0.0,2.0,18.0,0.0,19.733333,24.996667,...,76.126667,129.380000,1.370000,0.533333,428.766667,12.553333,108.433333,14.613333,18.767333,21.963333
3,2016-01-04,141638.0,5210.0,254.0,0.0,3.0,18.0,0.0,18.940000,24.516667,...,76.680000,138.363333,1.283333,0.533333,427.900000,12.453333,109.633333,14.363333,18.524333,21.756667
4,2016-01-05,126207.8,5387.0,254.0,0.0,4.0,18.0,0.0,18.176667,23.930000,...,76.080000,136.733333,1.263333,0.266667,445.766667,12.736667,76.466667,14.150000,18.305333,21.550000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,2020-09-24,134359.9,4775.0,297.0,0.0,0.0,14.0,0.0,22.750000,27.566667,...,78.190000,134.636667,1.466667,6.083333,381.700000,12.056667,230.700000,16.750000,20.304333,22.990000
1729,2020-09-25,126926.0,5039.0,297.0,0.0,0.0,15.0,0.0,22.473333,27.233333,...,77.693333,128.676667,1.473333,6.083333,389.066667,12.120000,208.433333,16.313333,20.044667,22.916667
1730,2020-09-26,110357.7,5565.0,294.5,0.0,0.0,16.0,0.0,22.146667,26.993333,...,76.980000,122.496667,1.476667,5.983333,408.100000,12.503333,179.400000,15.910000,19.772667,22.846667
1731,2020-09-27,620.0,5227.0,255.0,0.0,0.0,17.0,0.0,21.856667,26.816667,...,76.320000,132.170000,1.433333,0.683333,426.100000,12.906667,141.933333,15.586667,19.525000,22.660000


In [37]:
test.columns

Index(['date', '배추_거래량(kg)', '배추_가격(원/kg)_배추', 'rain_sum_1q_배추',
       'heavy_rain_count_1q_배추', 'low_temp_count_1q_배추',
       'middle_temp_count_1q_배추', 'high_temp_count_1q_배추', 'avg_temp_1q_배추',
       'avg_max_Temp_1q_배추',
       ...
       'avg_hum_12q_샤인마스캇', 'avg_widdir_12q_샤인마스캇', 'avg_wind_12q_샤인마스캇',
       'avg_rain_12q_샤인마스캇', 'avg_sun_Time_12q_샤인마스캇', 'avg_sun_Qy_12q_샤인마스캇',
       'avg_condens_Time_12q_샤인마스캇', 'avg_gr_Temp_12q_샤인마스캇',
       'avg_soil_Temp_12q_샤인마스캇', 'avg_soil_Wt_12q_샤인마스캇'],
      dtype='object', length=4579)

In [38]:
pum_df_list[0].shape
test.shape
crude_data = pd.read_csv('final_crude_data.csv', index_col= 0)

In [39]:
final_merge= pd.merge(test, crude_data, on = 'date', how = 'left')

In [40]:
final_merge.to_csv('final_data.csv', index = None)

In [41]:
# test_df = df21[df21.PUM_NM == '배추'].drop('KIND_NM', axis = 1)

In [42]:
# test_df.DAN_NM.unique()

In [43]:
# test_df[test_df.DAN_NM == 'ton(M/T)']

In [44]:
# test_df[test_df.DAN_NM == 'g']

In [45]:
# test_df[test_df.DAN_NM == '.']

In [46]:
# test_df.DAN_NM.unique()
#TOT_QTY: 총물량 (음수로 집계된 값은 거래 취소 내역)
# TOT_AMT: 총금액
# test_df2 = test_df.drop('DAN_NM', axis = 1)

In [47]:
# test_df3 = test_df2.groupby(['SALEDATE','PUM_NM'])['TOT_QTY','TOT_AMT'].sum().reset_index()
# test_df3

In [48]:
# train